In [ ]:
# !pip3 install torch

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd

In [ ]:
# get data
data = pd.read_csv("../us/data.csv", delimiter=",", names=["date", "temperature", "windspeed"], skiprows=1)
data = data.dropna()

In [ ]:
temperature = data["temperature"].values.reshape(-1, 1)
wind_speed = data["wind_speed"].values.reshape(-1, 1)
accidents = data["amount_of_accidents"].values.reshape(-1, 1)


# Standardize the input features
temperature_mean = temperature.mean()
temperature_std = temperature.std()

wind_speed_mean = wind_speed.mean()
wind_speed_std = wind_speed.std()

temperature_standardized = (temperature - temperature_mean) / temperature_std
wind_speed_standardized = (wind_speed - wind_speed_mean) / wind_speed_std

# Convert the standardized data to PyTorch tensors
X = torch.tensor(
    list(zip(temperature_standardized, wind_speed_standardized)),
    dtype=torch.float32,
)
y = torch.tensor(accidents, dtype=torch.float32)

# X.size(), y.size()

In [ ]:
# Define the linear regression model
class LinearRegression(nn.Module):
    def __init__(self):
        super(LinearRegression, self).__init__()
        self.linear = nn.Linear(2, 1)  # 2 input features and 1 output

    def forward(self, x):
        return self.linear(x)

In [ ]:
# Initialize the model
model = LinearRegression()
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

In [ ]:
current_temperature = 0
current_wind_speed = 0


new_data = torch.tensor([[current_temperature, current_temperature]], dtype=torch.float32)

# Standardize the new_data using the same mean and standard deviation as the training data
new_data_standardized = (new_data - torch.tensor([temperature_mean, wind_speed_mean])) / torch.tensor([temperature_std, wind_speed_std])

predicted = model(new_data_standardized)

print(f'Predicted amount of accidents: {predicted.item():.2f}')
